# 快速开始

点击顶部栏上的`🚀` -> `Binder`按钮能够在线运行此示例。

这个例子将展示：

- 如何通过节点连接到区块链
- 如何创建和使用`account`
- 如何与合约交互，如何发送交易

## 连接到节点

我们一般会通过节点连接到 Conflux 区块链。您可以按照[本教程](https://developer.confluxnetwork.org/conflux-doc/docs/get_started/)在本地运行节点。为了演示的方便，在这个例子中，我们连接到的节点为[Conflux 公共 RPC](https://developer.confluxnetwork.org/sdks-and-tools/en/conflux_rpcs)来连接到测试网。

In [1]:
from conflux_web3 import Web3

w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))
assert w3.is_connected()

### 创建一个帐户

在 web3 中，拥有一个帐户往往意味着知道一个**秘密值**，该值也被称为账户的**私钥/密钥**。也就是说，知道私钥的人也就控制了账户。

在SDK中，我们可以使用`w3.account.create()`创建一个新帐户。该函数使用一个随机生成的密钥创建帐户对象。

SDK 也支持使用`w3.account.from_key("0x....")`来使用私钥（**但不要在不安全的环境中使用私钥** ）

In [2]:
acct = w3.account.create()
print(f"account address: {acct.address}")
# 注意: 不要在生产环境中运行下面这行，私钥应该保密
print(f"account secret key: {acct.key.hex()}")
balance = w3.cfx.get_balance(acct.address)
# 随机账户，因此余额为 0
assert balance == 0
print(f"balance for {acct.address}: {balance}")

account address: cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb
account secret key: 0x54d957b2485980fc2119ccce6480ad0d53219161595424fbf7dc452d59c6bc82
balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 0 Drip


## 通过测试网水龙头合约领取 CFX

因为账户的余额是`0`，所以该账户无法支付发送交易所需的交易费（gas）。但是，Conflux 的[赞助机制](https://forum.conflux.fun/t/conflux-sponsorship-mechanism/12764)允许用户在不支付 gas 的情况下与智能合约进行交互，因此我们可以从[测试网水龙头合约](https://testnet.confluxscan.net/address/cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61)中领取 CFX。

> 智能合约是什么：智能合约是部署在区块链网络上的程序。它提供了用于执行特定逻辑的接口。在下面示例中，调用水龙头合约的 `claimCfx` 方法将为您提供 1000 个测试网 CFX。

In [3]:
# 首先设置 `w3.cfx.default_account` 为 `acct`,
# 这样交易能够被自动签名发送。
w3.cfx.default_account = acct

# 与测试网水龙头合约交互
faucet = w3.cfx.contract(name="Faucet")
tx_hash = faucet.functions.claimCfx().transact()

print(f"tx hash is: {tx_hash.hex()}\n"
      f"confluxscan link: https://testnet.confluxscan.net/transaction/{tx_hash.hex()}")

tx hash is: 0xf174cf4be19b33da58504ef850bfd055bd031324135093276abbee1c34df854d
conflux scan link: https://testnet.confluxscan.net/transaction/0xf174cf4be19b33da58504ef850bfd055bd031324135093276abbee1c34df854d


In [4]:
# 在 Conflux, 交易上链 5 个 epoch 之后才会被执行
# `tx_hash.executed()` 与 `w3.cfx.wait_for_transaction_receipt(tx_hash)` 是等价的
tx_hash.executed()
# Drip 与 CFX 是 Conflux 区块链中的代币单位
# 1 CFX = 10**18 Drip
print(f"balance for {acct.address}: {w3.cfx.get_balance(acct.address).to('CFX')}")

balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 1000 CFX


## 进行 CFX 转账

现在我们能够支付交易费了。接下来我们可以进行转账操作，例如将 1 个 CFX 发送到零地址。

In [5]:
# Now acct has CFX
# send 1 CFX to zero address
w3.cfx.send_transaction({
    "to": w3.address.zero_address(),
    "value": 10**18,
}).executed()
print(f"balance for {acct.address}: {w3.cfx.get_balance(acct.address).to('CFX')}")

balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 998.999979 CFX
